In [19]:
import torch
from datasets.mnist import MNIST
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import torchvision
import os
from ipywidgets import interact, FloatSlider
import pandas as pd
import numpy as np


from nn.enforced_ae import EnforcedAE
from nn.ae import AE
from train import train
from schedule import cosine_schedule


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   


In [2]:
train_dataset = MNIST(root='../../datasets', split='train', download=False, device=device, normalize=False, augment=True)
val_dataset = MNIST(root='../../datasets', split='val', download=False, device=device, normalize=False)
test_dataset = MNIST(root='../../datasets', split='test', download=False, device=device, normalize=False)

In [23]:
def train_model(model, experiment_name, trial_name):
    optimiser = torch.optim.AdamW(model.parameters(), lr=1e-3)
    num_epochs = 200
    batch_size = 256
    compute_dtype = torch.bfloat16

    hyperparams = {
        'lr': cosine_schedule(base=1e-3, end=1e-4, T=num_epochs, warmup=10, flat_end=10),
        'wd': cosine_schedule(base=0.004, end=0.1, T=num_epochs)
    }

    trial_log_dir = f'out/logs/{experiment_name}/{trial_name}'
    run_no = 0
    while os.path.exists(trial_log_dir + f'/run_{run_no}'):
        run_no += 1
    writer = SummaryWriter(trial_log_dir + f'/run_{run_no}')
    save_dir = f'out/models/{experiment_name}/{trial_name}/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_dir=save_dir + f'run_{run_no}.pt'


    train(model, train_dataset, val_dataset, optimiser, num_epochs=num_epochs, batch_size=batch_size, writer=writer, compute_dtype=compute_dtype, save_dir=save_dir, epoch_hyperparams=hyperparams)

KeyboardInterrupt: 

In [ ]:
experiment_name = 'base'
trial_name = 'ae_cnn20_schedules'
ae = AE(in_channels=1, z_dim=20, cnn=True).to(device)
train_model(ae, experiment_name, trial_name)
ae.load_state_dict(torch.load(f'out/models/{experiment_name}/{trial_name}/run_0.pt'))

In [ ]:
experiment_name = 'base'
trial_name = 'enforced_ae_cnn20_schedules'
enforced_ae = EnforcedAE(in_channels=1, z_dim=20, cnn=True).to(device)
train_model(enforced_ae, experiment_name, trial_name)
enforced_ae.load_state_dict(torch.load(f'out/models/{experiment_name}/{trial_name}/run_0.pt'))

In [ ]:
# Enforced AE Learning
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

images, _= next(iter(test_loader))
def plot_images_with_widgets(model, images):
    def update(rotation=0., translate_x=0., translate_y=0., scale=1., shear=0.):
        action = torch.tensor([rotation, translate_x, translate_y, scale, shear]).to(device)
        images_aug, actions = model.transform_images(images, action)
        images_hat = model(images, action)[0]

        # Plot original images
        fig, axes = plt.subplots(1, 3, figsize=(12, 6))
        axes[0].imshow(torchvision.utils.make_grid(images.cpu(), nrow=4, padding=2, normalize=True).permute(1, 2, 0))
        axes[0].set_title('Original Inputs')
        axes[0].axis('off')

        axes[1].imshow(torchvision.utils.make_grid(images_aug.cpu(), nrow=4, padding=2, normalize=True).permute(1, 2, 0))
        axes[1].set_title('Transformed Targets')
        axes[1].axis('off')

        axes[2].imshow(torchvision.utils.make_grid(images_hat.cpu(), nrow=4, padding=2, normalize=True).permute(1, 2, 0))
        axes[2].set_title('Output Predictions')
        axes[2].axis('off')

        plt.show()
    
    interact(update, 
             rotation=FloatSlider(min=-1.0, max=1.0, step=0.1, value=0.),
             translate_x=FloatSlider(min=-1.0, max=1.0, step=0.1, value=0.),
             translate_y=FloatSlider(min=-1.0, max=1.0, step=0.1, value=0.),
             scale=FloatSlider(min=-1.0, max=1.0, step=0.1, value=0.),
             shear=FloatSlider(min=-1.0, max=1.0, step=0.1, value=0.))

plot_images_with_widgets(enforced_ae, images)

In [ ]:
def play_with_latents(model, dataset, labelled=False):
    idx = torch.randint(0, len(dataset), (1,))
    image = dataset[idx][0]
    z = model.infer(image)
    def update(z0=0., z1=0., z2=0., z3=0., z4=0., z5=0., z6=0., z7=0., z8=0., z9=0.):
        z[:, :10] = torch.tensor([z0, z1, z2, z3, z4, z5, z6, z7, z8, z9])
        x_hat = model.decode(z).detach()
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        axes[0].imshow(image.cpu().squeeze(0).permute(1, 2, 0), cmap='gray')
        axes[0].set_title('Original Image')
        axes[0].axis('off')

        axes[1].imshow(x_hat.cpu().squeeze(0).permute(1, 2, 0), cmap='gray')
        axes[1].set_title('Reconstructed Image')
        axes[1].axis('off')

        plt.show()

    sliders = {
        
        'z0': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,0].item(), description='(rotation) z0' if labelled else 'z0'),
        'z1': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,1].item(), description='(x offset) z1' if labelled else 'z1'),
        'z2': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,2].item(), description='(y offset) z2' if labelled else 'z2'),
        'z3': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,3].item(), description='(scale) z3' if labelled else 'z3'),
        'z4': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,4].item(), description='(shear) z4' if labelled else 'z4'),
        'z5': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,5].item()),
        'z6': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,6].item()),
        'z7': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,7].item()),
        'z8': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,8].item()),
        'z9': FloatSlider(min=-3.0, max=3.0, step=0.01, value=z[:,9].item()),
    }

    interact(update, **sliders)


In [ ]:
play_with_latents(ae, test_dataset, labelled=False)

In [ ]:
play_with_latents(enforced_ae, test_dataset, labelled=True)